In [5]:
import gym
import numpy as np
import ray
from really import SampleManager
from gridworlds import GridWorld
#%%time
import random
import os
#from really import SampleManager  # important !!
from really.utils import (
    dict_to_dict_of_datasets,
)

In [6]:
class TabularQ(object):

    def __init__(self, h, w, action_space):

        self.action_space = action_space
        #print(h,w)
        self.q_table = np.zeros((h, w, 4))


        #self.q_table.fill(0)


        #print(self.q_table)
        #print(self.q_table[2, 2])
        #q_table[0][3]

        ## # TODO:
        pass

    def __call__(self, state):
        ## # TODO:

        output = {}
        #print(state)
        #print(state[0][0], state[0][1])
        #print(self.q_table[2, 2])
        a = int(state[0][0])
        b = int(state[0][1])
        #print(a, b)
        #print(np.asmatrix(self.q_table[a, b]))

        output["q_values"] = np.asmatrix(self.q_table[a, b]) # achtung vllt []


        #output["q_values"] = np.random.normal(size=(1, self.action_space))
        #print(output["q_values"])
        #print(np.random.normal(size=(1, self.action_space)))

        return output

    # # TODO:
    def get_weights(self):

        #output =

        return self.q_table


    def set_weights(self, q_vals):

        print(q_vals)
        self.q_table = q_vals
        pass

In [7]:
ray.init(log_to_driver=False)


2021-02-25 19:40:00,070	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.178.21',
 'raylet_ip_address': '192.168.178.21',
 'redis_address': '192.168.178.21:6379',
 'object_store_address': '/tmp/ray/session_2021-02-25_19-39-59_490613_70820/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-25_19-39-59_490613_70820/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-02-25_19-39-59_490613_70820',
 'metrics_export_port': 63681,
 'node_id': '62b239937b20fba2e2b697f96a5428452672d34817dca2f83a67759e'}

In [8]:
action_dict = {0: "UP", 1: "RIGHT", 2: "DOWN", 3: "LEFT"}

env_kwargs = {
    "height": 3,
    "width": 4,
    "action_dict": action_dict,
    "start_position": (2, 0),
    "reward_position": (0, 3),
}

# you can also create your environment like this after installation: env = gym.make('gridworld-v0')
env = GridWorld(**env_kwargs)

model_kwargs = {"h": env.height, "w": env.width, "action_space": 4}

kwargs = {
    "model": TabularQ,
    "environment": GridWorld,
    "num_parallel": 2,
    "total_steps": 5,
    "model_kwargs": model_kwargs

    # and more
}

# initilize
manager = SampleManager(**kwargs)
#print(manager.sample(10))

manager.get_data(do_print=True)

# where to save your results to: create this directory in advance!
saving_path = os.getcwd() + "../Homework/A1/progress_test"

buffer_size = 5000
test_steps = 1000
epochs = 5
sample_size = 1000
optim_batch_size = 8
saving_after = 5
alpha = 0.2
gamma = 0.85

# keys for replay buffer -> what you will need for optimization
optim_keys = ["state", "action", "reward", "state_new", "not_done"]

# initialize buffer
manager.initilize_buffer(buffer_size, optim_keys)

# initilize progress aggregator
manager.initialize_aggregator(
    path=saving_path, saving_after=5, aggregator_keys=["loss", "time_steps"]
)

iteration: 0, storing results of 1 runners


In [9]:
# initial testing:
print("test before training: ")
manager.test(test_steps, do_print=True)

# get initial agent
agent = manager.get_agent()

test before training: 
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
Episodes finished after a mean of 42.33 timesteps
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]


In [ ]:
for e in range(epochs):
    print("collecting experience..")
    data = manager.get_data()
    manager.store_in_buffer(data)

    # sample data to optimize on from buffer
    sample_dict = manager.sample(sample_size)
    print(f"collected data for: {sample_dict.keys()}")
    # create and batch tf datasets
    data_dict = dict_to_dict_of_datasets(sample_dict, batch_size=optim_batch_size)
    state, next_state, action, reward = data_dict['state'], data_dict['state_new'], data_dict['action'], data_dict['reward']  
    q_table = agent.get_weights().copy()
    for s_batch, s_next_batch, a_batch, r_batch in zip(state, next_state, action, reward):
        
        old_value_batch = [q_table[s[0], s[1], a] for s, a in zip(s_batch, a_batch)]
        next_max_batch = [np.max(q_table[s_next[0], s_next[1]]) for s_next in s_next_batch]
        new_values = [old_value + alpha * ((r.numpy()) + gamma * next_max - old_value)
                     for old_value, r, next_max in zip(old_value_batch, r_batch, next_max_batch)]
        breakpoint()
        q_table[s[0][0], s[0][1], a[0]] = new_value
        if r.numpy()[0] != 0:
            pass
            #breakpoint()
    # set new weights
    #breakpoint()
    manager.set_agent(q_table)
    # get new weights
    agent = manager.get_agent()

collecting experience..
collected data for: dict_keys(['state', 'action', 'reward', 'state_new', 'not_done'])
> <ipython-input-12-58d63b21dbcb>(20)<module>()
     18                      for old_value, r, next_max in zip(old_value_batch, r_batch, next_max_batch)]
     19         breakpoint()
---> 20         q_table[s[0][0], s[0][1], a[0]] = new_value
     21         if r.numpy()[0] != 0:
     22             pass

ipdb> print(s_batch)
tf.Tensor(
[[2 0]
 [1 0]
 [2 0]
 [0 1]
 [0 1]
 [0 1]
 [2 1]
 [1 0]], shape=(8, 2), dtype=int64)
ipdb> np.concatenate
<function concatenate at 0x7ff74c289430>
ipdb> np.concatenate(s_batch, a, axis=1)
*** TypeError: concatenate() got multiple values for argument 'axis'
ipdb> np.concatenate((s_batch, a_batch), axis=1)
*** ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)
ipdb> np.concatenate((s_batch, a_batch), axis=1)
*** ValueError: all the input arra

In [54]:
manager.test(
       max_steps=100,
        test_episodes=10,
        render=True,
        do_print=True,
        evaluation_measure="time_and_reward",
    )

[[[ 4.43705312  5.2200625   5.2200625   4.43705312]
  [ 5.2200625   6.14125     5.2200625   4.43705312]
  [ 6.14125     7.225       7.225       5.2200625 ]
  [ 7.225       7.225       8.5         6.14125   ]]

 [[ 4.43705312  5.2200625   6.14125     5.2200625 ]
  [ 0.          0.          0.          0.        ]
  [ 6.14125     8.5         8.5         7.225     ]
  [ 7.2236474   8.4999994  10.          7.225     ]]

 [[ 5.2200625   7.225       6.14125     6.14125   ]
  [ 7.225       8.5         7.225       6.14125   ]
  [ 7.225      10.          8.5         7.225     ]
  [ 0.          0.          0.          0.        ]]]
Episodes finished after a mean of 4.0 timesteps
Episodes finished after a mean of 2.0 accumulated reward


([4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0])